In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.redfinclean
df = pd.DataFrame(list(db.redfin.find().limit(10000)))

In [12]:
sel_cols = ['period_begin',  
'period_end', 
'region_name' ,
'region_type',
'total_homes_sold',
'median_sale_price',
# 'median_sale_ppsf',
'inventory'
]

In [13]:
df_reduced = df[sel_cols]

In [14]:
# df_reduced["period_begin"].to_datetime()
# 
begin_dates = pd.to_datetime(df_reduced["period_begin"], infer_datetime_format=True)
df_reduced["year"] = begin_dates.dt.strftime("%Y")
#df_reduced["median_sale_ppsf"].round(2)
df_reduced["year_month"] = begin_dates.dt.strftime("%Y-%m")

states = []
for region in df_reduced["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
df_reduced["state"] = states

C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [15]:
df_counties = df_reduced.loc[df_reduced["region_type"] != "metro"]

In [16]:
df_counties.head()

,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,inventory,year,year_month,state
0,2019-09-09,2019-09-15,"Haywood County, NC",county,34.0,240000.0,571.0,2019,2019-09,NC
1,2018-11-05,2018-11-11,"Eaton County, MI",county,19.0,187500.0,316.0,2018,2018-11,MI
2,2020-07-20,2020-07-26,"Greenville County, SC",county,207.0,255000.0,2561.0,2020,2020-07,SC
3,2019-02-04,2019-02-10,"Dane County, WI",county,65.0,274000.0,1542.0,2019,2019-02,WI
4,2017-05-01,2017-05-07,"Aroostook County, ME",county,8.0,58700.0,448.0,2017,2017-05,ME


In [17]:
df_counties.fillna('', inplace=True)
df_counties

C:\Users\kyucz\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,inventory,year,year_month,state
0,2019-09-09,2019-09-15,"Haywood County, NC",county,34.0,240000.0,571.0,2019,2019-09,NC
1,2018-11-05,2018-11-11,"Eaton County, MI",county,19.0,187500.0,316.0,2018,2018-11,MI
2,2020-07-20,2020-07-26,"Greenville County, SC",county,207.0,255000.0,2561.0,2020,2020-07,SC
3,2019-02-04,2019-02-10,"Dane County, WI",county,65.0,274000.0,1542.0,2019,2019-02,WI
4,2017-05-01,2017-05-07,"Aroostook County, ME",county,8.0,58700.0,448.0,2017,2017-05,ME
...,...,...,...,...,...,...,...,...,...,...
9994,2017-02-13,2017-02-19,"Grafton County, NH",county,22.0,163000.0,754.0,2017,2017-02,NH
9996,2017-01-23,2017-01-29,"Harlan County, KY",county,3.0,55320.0,52.0,2017,2017-01,KY
9997,2020-03-30,2020-04-05,"Duval County, FL",county,364.0,216000.0,4119.0,2020,2020-03,FL
9998,2020-05-25,2020-05-31,"Hudson County, NJ",county,44.0,560500.0,1648.0,2020,2020-05,NJ


In [25]:
df_counties.reset_index(inplace = True)
data_dict = df_counties.to_dict("records")

collection.insert_many(data_dict)